In [1]:
import pickle 

with open('../../data/Hazumi_features/Hazumi1911_features.pkl', mode='rb') as f:
    SS, TS, SP, TP, text, audio, visual, bio, vid = pickle.load(f, encoding='utf-8')

In [2]:
for id in vid:
    count = 0
    for ss, ts in zip(SS[id], TS[id]):
        if ss == 0 and (ts == 1 or ts == 2):
            count += 1
        elif ts == 0 and (ss == 1 or ss == 2):
            count += 1 
    print(round(count/len(SS[id]),2))

0.24
0.39
0.27
0.51
0.56
0.23
0.28
0.07
0.06
0.36
0.36
0.02
0.01
0.02
0.14
0.33
0.2
0.13
0.08
0.06
0.01
0.14
0.06
0.37
0.22
0.08


In [3]:
results = []
for id in vid:
    count = 0
    for ss, ts in zip(SS[id], TS[id]):
        if ss != ts:
            count += 1
    results.append(round(count/len(SS[id]),2))
    print(id, round(count/len(SS[id]),2), count, len(SS[id]))
print(sum(results) / len(results))
min(results)

1911F2001 0.42 30 71
1911F2002 0.78 88 113
1911F3001 0.53 39 74
1911F3002 0.73 56 77
1911F3003 0.71 70 98
1911F4001 0.65 63 97
1911F4002 0.62 68 110
1911F4003 0.26 23 90
1911F5001 0.42 46 110
1911F5002 0.64 72 113
1911F6001 0.69 56 81
1911F6002 0.33 28 84
1911F6003 0.3 25 84
1911F7002 0.39 38 97
1911M2001 0.41 39 96
1911M2002 0.45 45 101
1911M2003 0.56 55 98
1911M4001 0.44 42 95
1911M4002 0.3 32 107
1911M5001 0.35 36 104
1911M5002 0.13 10 77
1911M6001 0.35 33 94
1911M6002 0.24 13 54
1911M6003 0.51 56 110
1911M7001 0.48 48 99
1911M7002 0.32 34 105
0.46192307692307694


0.13

In [4]:
import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("r-yanagimoto/sentiment")

summary_list, config_list, name_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "name": name_list
    })

runs_df.to_csv("project.csv")

In [5]:
runs_df

,summary,config,name
0,"{'_trn loss': 1.0726, '_val loss': 1.1104, '_t...","{'D_h1': 256, 'D_h2': 64, 'modal': 'av', 'epoc...",1911M7002
1,"{'_trn loss': 1.0228, '_val loss': 1.1036, '_t...","{'D_h1': 256, 'D_h2': 64, 'modal': 'v', 'epoch...",1911M7002
2,"{'_wandb': {'runtime': 50}, '_runtime': 51.682...","{'D_h1': 256, 'D_h2': 64, 'modal': 'av', 'epoc...",1911M7001
3,"{'_step': 124, '_wandb': {'runtime': 65}, '_ru...","{'D_h1': 256, 'D_h2': 64, 'modal': 'av', 'epoc...",1911M6003
4,"{'_timestamp': 1683791180.5391266, 'acc': 0.92...","{'D_h1': 256, 'D_h2': 64, 'modal': 'av', 'epoc...",1911M6002
...,...,...,...
619,"{'_step': 33, '_wandb': {'runtime': 26}, '_run...","{'bio': False, 'D_h1': 256, 'D_h2': 64, 'third...",1911F2001
620,"{'_step': 41, '_wandb': {'runtime': 17}, '_run...","{'bio': False, 'D_h1': 256, 'D_h2': 64, 'third...",1911F3002
621,"{'tst loss': 1.1007, '_trn loss': 1.0017, '_va...","{'bio': False, 'D_h1': 256, 'D_h2': 64, 'third...",1911F3001
622,"{'_runtime': 38.01271438598633, 'tst loss': 1....","{'bio': False, 'D_h1': 256, 'D_h2': 64, 'third...",1911F2002
